In [ ]:
# !pip install  attrs 
# !pip install scrapy 
# !pip install --upgrade Twisted



In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # Set interactive shell to display all node outputs
import json,logging
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

In [2]:
# List of websites to scrape
WebSite = ['http://quotes.toscrape.com/page/1/']

# Pipeline to write scraped data to a JSON Lines file
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('ScrapFile.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        # Convert the item to JSON and write to the file
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

# Spider to scrape quotes from a website
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    # Start URLs for the spider
    start_urls = WebSite

    # Custom settings for the spider
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,  # Set log level to WARNING
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},  # Use the custom JSON writer pipeline
        'FEED_FORMAT': 'json',  # Set the feed format to JSON
        'FEEDS': {'quates.json': {'format': 'json'}}} # Specify the output file for the feed
    

    # Parse method to extract data from the web page
    def parse(self, response):
        for quote in response.css('div.quote'):
            # Yield a dictionary containing quote information
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract()
            }

# Create a CrawlerProcess with custom user agent
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36'
})

# Start the crawler with the QuotesSpider
process.crawl(QuotesSpider)
process.start()


2023-11-25 10:24:37 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2023-11-25 10:24:37 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.8.8 (default, Apr 13 2021, 19:58:26) - [GCC 7.3.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Linux-6.2.0-37-generic-x86_64-with-glibc2.10
2023-11-25 10:24:37 [py.warnings] WARNING: /home/bilel/anaconda3/lib/python3.8/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for informatio

In [ ]:
# !pip freeze | grep ipyth >> requirements.txt



In [5]:
data = pd.read_json('quates.json')
print(data.shape)
data.head()


(10, 3)


,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"


In [6]:
data.to_csv('Scrapped_Data.csv')

In [7]:
data

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
5,“Try not to become a man of success. Rather be...,Albert Einstein,"[adulthood, success, value]"
6,“It is better to be hated for what you are tha...,André Gide,"[life, love]"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"[edison, failure, inspirational, paraphrased]"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,[misattributed-eleanor-roosevelt]
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"[humor, obvious, simile]"
